## Load module and data, initiate H2O session

In [3]:
import pandas as pd
import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.automl import H2OAutoML

In [4]:
# Initiate session
h2o.init()
h2o.cluster().show_status()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,05 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,3 months and 26 days !!!
H2O cluster name:,H2O_from_python_yuqingz_zv338e
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


H2O cluster uptime:,05 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,3 months and 26 days !!!
H2O cluster name:,H2O_from_python_yuqingz_zv338e
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [5]:
# Read in saved data
df_trn = pd.read_pickle("trn.pkl")
df_tst = pd.read_pickle("tst.pkl")
print(df_trn.shape); print(df_tst.shape)
print(type(df_trn)); print(type(df_tst))

(8625, 18)
(3699, 18)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [6]:
df_trn.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
2476,5,626.000000,1,66.0,77,2660.150000,0.0,0.008228,41.925190,0.0,May,1,1,3,2,Returning_Visitor,False,False
582,1,43.000000,0,0.0,37,1505.166667,0.0,0.005263,0.000000,0.0,Mar,2,2,4,8,New_Visitor,False,False
10029,0,0.000000,0,0.0,9,258.291667,0.0,0.022222,0.000000,0.0,Nov,2,5,3,2,Returning_Visitor,False,False
1154,0,0.000000,0,0.0,1,0.000000,0.2,0.200000,0.000000,0.0,Mar,1,1,1,9,Returning_Visitor,True,False
1579,11,348.654762,1,11.0,64,1229.289286,0.0,0.002494,54.951269,0.0,Mar,2,2,1,2,New_Visitor,False,False


### Convert DataFrame to H2OFrame

In [7]:
hf_trn = h2o.H2OFrame(df_trn)
hf_tst = h2o.H2OFrame(df_tst)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


### Further split training set into 80% train, 20% validation

In [9]:
train, val = hf_trn.split_frame(ratios=[0.2], seed=123)
print(train.shape)
print(val.shape)
test = hf_tst

(1753, 18)
(6872, 18)


## Training Random Forest

In [12]:
rf = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10)
rf.train(y="Revenue", training_frame=train)
rf_perf_val = rf.model_performance(test_data=val)
print(rf_perf_val)

drf Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.07004523598207531
RMSE: 0.2646606052703638
LogLoss: 0.2881540674930204
Mean Per-Class Error: 0.152942842882003
AUC: 0.9200597100999609
pr_auc: 0.7162223761504781
Gini: 0.8401194201999218
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3733333331346512: 


,False,True,Error,Rate
False,5464.0,396.0,0.0676,(396.0/5860.0)
True,306.0,706.0,0.3024,(306.0/1012.0)
Total,5770.0,1102.0,0.1022,(702.0/6872.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3733333,0.6679281,73.0
max f2,0.1600000,0.7386926,127.0
max f0point5,0.5733333,0.6851230,38.0
max accuracy,0.4400000,0.9045402,59.0
max precision,1.0,1.0,0.0
max recall,0.0,1.0,183.0
max specificity,1.0,1.0,0.0
max absolute_mcc,0.4400000,0.6089544,59.0
max min_per_class_accuracy,0.2000000,0.8452218,115.0
max mean_per_class_accuracy,0.1600000,0.8470572,127.0


Gains/Lift Table: Avg response rate: 14.73 %, avg score: 15.62 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0122235,0.9,6.3863166,6.3863166,0.9404762,0.9319048,0.9404762,0.9319048,0.0780632,0.0780632,538.6316582,538.6316582
,2,0.0240105,0.84,6.1198458,6.2555037,0.9012346,0.8582716,0.9212121,0.8957576,0.0721344,0.1501976,511.9845801,525.5503653
,3,0.0325960,0.8,5.9848597,6.1842180,0.8813559,0.8113559,0.9107143,0.8735268,0.0513834,0.2015810,498.4859650,518.4217956
,4,0.0405995,0.76,5.3089472,6.0116735,0.7818182,0.7725455,0.8853047,0.8536201,0.0424901,0.2440711,430.8947179,501.1673538
,5,0.0504948,0.72,5.4923274,5.9098996,0.8088235,0.7314216,0.8703170,0.8296734,0.0543478,0.2984190,449.2327366,490.9899648
,6,0.1049185,0.54,4.1941409,5.0198944,0.6176471,0.6187433,0.7392510,0.7202589,0.2282609,0.5266798,319.4140897,401.9894415
,7,0.1527939,0.4,3.0753391,4.4106004,0.4528875,0.4574823,0.6495238,0.6379222,0.1472332,0.6739130,207.5339092,341.0600414
,8,0.2076542,0.28,1.8912572,3.7450136,0.2785146,0.3274889,0.5515067,0.5559087,0.1037549,0.7776680,89.1257169,274.5013586
,9,0.3134459,0.14,1.1768979,2.8782447,0.1733150,0.1928198,0.4238626,0.4333620,0.1245059,0.9021739,17.6897858,187.8244722
,10,0.4202561,0.0800000,0.4070608,2.2501807,0.0599455,0.0986494,0.3313712,0.3482931,0.0434783,0.9456522,-59.2939225,125.0180658


In [13]:
rf_perf_tst = rf.model_performance(test_data=test)
print(rf_perf_tst)


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.08013629743773679
RMSE: 0.2830835520438035
LogLoss: 0.36692275849042866
Mean Per-Class Error: 0.15626604173641334
AUC: 0.9050706500656283
pr_auc: 0.699466336765143
Gini: 0.8101413001312565
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.29000000000000004: 


,False,True,Error,Rate
False,2758.0,332.0,0.1074,(332.0/3090.0)
True,149.0,460.0,0.2447,(149.0/609.0)
Total,2907.0,792.0,0.13,(481.0/3699.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2900000,0.6566738,74.0
max f2,0.2000000,0.7489879,97.0
max f0point5,0.6,0.6658098,29.0
max accuracy,0.52,0.8853744,37.0
max precision,1.0,1.0,0.0
max recall,0.0,1.0,144.0
max specificity,1.0,1.0,0.0
max absolute_mcc,0.2300000,0.5877024,89.0
max min_per_class_accuracy,0.2000000,0.8368932,97.0
max mean_per_class_accuracy,0.2000000,0.8437340,97.0


Gains/Lift Table: Avg response rate: 16.46 %, avg score: 16.37 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0127061,0.92,5.8154282,5.8154282,0.9574468,0.9442553,0.9574468,0.9442553,0.0738916,0.0738916,481.5428152,481.5428152
,2,0.0221682,0.86,5.5532723,5.7035324,0.9142857,0.8748571,0.9390244,0.9146341,0.0525452,0.1264368,455.3272343,470.3532380
,3,0.0327115,0.8200000,5.2951876,5.5719171,0.8717949,0.8310256,0.9173554,0.8876860,0.0558292,0.1822660,429.5187571,457.1917111
,4,0.0400108,0.7610667,4.4991790,5.3762149,0.7407407,0.7917284,0.8851351,0.8701802,0.0328407,0.2151067,349.9178982,437.6214885
,5,0.0513652,0.74,5.2061928,5.3386311,0.8571429,0.7528571,0.8789474,0.8442456,0.0591133,0.2742200,420.6192822,433.8631060
,6,0.1027305,0.56,3.6763028,4.5074669,0.6052632,0.6378509,0.7421053,0.7410482,0.1888342,0.4630542,267.6302826,350.7466943
,7,0.1530143,0.4200000,2.8083585,3.9491027,0.4623656,0.4748208,0.6501767,0.6535601,0.1412151,0.6042693,180.8358494,294.9102682
,8,0.2027575,0.3200000,2.5087813,3.5957438,0.4130435,0.3594511,0.592,0.5814053,0.1247947,0.7290640,150.8781324,259.5743842
,9,0.3087321,0.1600000,1.3480321,2.8242000,0.2219388,0.2238605,0.4649737,0.4586754,0.1428571,0.8719212,34.8032070,182.4200047
,10,0.4252501,0.0800000,0.5355171,2.1971038,0.0881671,0.1040580,0.3617292,0.3615107,0.0623974,0.9343186,-46.4482873,119.7103837


## AutoML

In [15]:
aml = H2OAutoML(max_runtime_secs=60, seed=1, 
                 project_name="purchasing_intention")
aml.train(y="Revenue", training_frame=train, validation_frame=val)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [16]:
aml.leaderboard.head()

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_grid_1_AutoML_20190924_171759_model_2,0.926013,0.237955,0.190624,0.267508,0.0715606
XGBoost_grid_1_AutoML_20190924_171643_model_2,0.926013,0.237955,0.190624,0.267508,0.0715606
XGBoost_2_AutoML_20190924_171759,0.924836,0.24189,0.16618,0.268566,0.0721277
XGBoost_2_AutoML_20190924_171643,0.924836,0.24189,0.16618,0.268566,0.0721277
XGBoost_1_AutoML_20190924_171643,0.924468,0.241069,0.151248,0.269428,0.0725913
XGBoost_1_AutoML_20190924_171759,0.924468,0.241069,0.151248,0.269428,0.0725913
XGBoost_grid_1_AutoML_20190924_171643_model_3,0.923955,0.241877,0.188491,0.269932,0.0728634
XGBoost_3_AutoML_20190924_171643,0.922762,0.242306,0.154701,0.268951,0.0723348
XGBoost_3_AutoML_20190924_171759,0.922762,0.242306,0.154701,0.268951,0.0723348
XGBoost_grid_1_AutoML_20190924_171759_model_1,0.922526,0.24313,0.15679,0.271465,0.0736934


In [17]:
pred = aml.predict(test)
perf = aml.leader.model_performance(test)
perf

xgboost prediction progress: |████████████████████████████████████████████| 100%

ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.07894044866708652
RMSE: 0.28096342941223956
LogLoss: 0.25589810172561717
Mean Per-Class Error: 0.15539161764471432
AUC: 0.9200147730110904
pr_auc: 0.7176737219848535
Gini: 0.8400295460221807
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.219113290309906: 


,False,True,Error,Rate
False,2742.0,348.0,0.1126,(348.0/3090.0)
True,140.0,469.0,0.2299,(140.0/609.0)
Total,2882.0,817.0,0.1319,(488.0/3699.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2191133,0.6577840,240.0
max f2,0.1045433,0.7525224,293.0
max f0point5,0.7235769,0.6805808,76.0
max accuracy,0.5782063,0.8888889,118.0
max precision,0.9658068,1.0,0.0
max recall,0.0172373,1.0,376.0
max specificity,0.9658068,1.0,0.0
max absolute_mcc,0.2047483,0.5888622,246.0
max min_per_class_accuracy,0.1453363,0.8407225,272.0
max mean_per_class_accuracy,0.1045433,0.8446084,293.0


Gains/Lift Table: Avg response rate: 16.46 %, avg score: 15.88 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100027,0.9304672,5.7455732,5.7455732,0.9459459,0.9457239,0.9459459,0.9457239,0.0574713,0.0574713,474.5573159,474.5573159
,2,0.0200054,0.9036663,5.9097324,5.8276528,0.9729730,0.9169344,0.9594595,0.9313292,0.0591133,0.1165846,490.9732392,482.7652776
,3,0.0300081,0.8702396,5.4172547,5.6908534,0.8918919,0.8873314,0.9369369,0.9166633,0.0541872,0.1707718,441.7254693,469.0853415
,4,0.0400108,0.8444433,4.7606178,5.4582945,0.7837838,0.8555812,0.8986486,0.9013927,0.0476190,0.2183908,376.0617761,445.8294501
,5,0.0500135,0.8130750,4.5964585,5.2859273,0.7567568,0.8299520,0.8702703,0.8871046,0.0459770,0.2643678,359.6458527,428.5927307
,6,0.1000270,0.6156302,3.9069897,4.5964585,0.6432432,0.7071094,0.7567568,0.7971070,0.1954023,0.4597701,290.6989748,359.6458527
,7,0.1500406,0.4091056,3.0861936,4.0930369,0.5081081,0.5063101,0.6738739,0.7001747,0.1543514,0.6141215,208.6193583,309.3036879
,8,0.2000541,0.2569377,2.1340700,3.6032952,0.3513514,0.3237922,0.5932432,0.6060791,0.1067323,0.7208539,113.4070031,260.3295167
,9,0.3000811,0.1094087,1.5430968,2.9165624,0.2540541,0.1771171,0.4801802,0.4630917,0.1543514,0.8752053,54.3096791,191.6562375
,10,0.4001081,0.0510846,0.5745573,2.3310611,0.0945946,0.0731284,0.3837838,0.3656009,0.0574713,0.9326765,-42.5442684,133.1061110
